# Лабораторная работа 7

In [ ]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv('credit_scoring.csv')
df

,client_id,DIR,Age,NumLoans,NumRealEstateLoans,NumDependents,Num30-59Delinquencies,Num60-89Delinquencies,Income,BalanceToCreditLimit,Delinquent90
0,0,0.496289,49.1,13,0,0.0,2,0,5298.360639,0.387028,0
1,1,0.433567,48.0,9,2,2.0,1,0,6008.056256,0.234679,0
2,2,2206.731199,55.5,21,1,NaN,1,0,NaN,0.348227,0
3,3,886.132793,55.3,3,0,0.0,0,0,NaN,0.971930,0
4,4,0.000000,52.3,1,0,0.0,0,0,2504.613105,1.004350,0
...,...,...,...,...,...,...,...,...,...,...,...
74995,74995,0.169463,67.6,5,1,0.0,0,0,20129.654428,0.074689,0
74996,74996,2085.662215,43.2,1,1,0.0,0,0,NaN,1.005390,0
74997,74997,0.118935,23.0,4,0,0.0,2,0,2019.117329,0.384511,0
74998,74998,0.203414,46.3,14,2,0.0,0,0,16823.909642,0.524093,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75000 entries, 0 to 74999
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   client_id              75000 non-null  int64  
 1   DIR                    75000 non-null  float64
 2   Age                    75000 non-null  float64
 3   NumLoans               75000 non-null  int64  
 4   NumRealEstateLoans     75000 non-null  int64  
 5   NumDependents          73084 non-null  float64
 6   Num30-59Delinquencies  75000 non-null  int64  
 7   Num60-89Delinquencies  75000 non-null  int64  
 8   Income                 60153 non-null  float64
 9   BalanceToCreditLimit   75000 non-null  float64
 10  Delinquent90           75000 non-null  int64  
dtypes: float64(5), int64(6)
memory usage: 6.3 MB


данные были взяты из лабораторной 4, пропусков и анамалий в них нет

In [ ]:
df = df.drop(columns=["client_id"])

делим на тест, валидацию и трейн

In [ ]:
X = df.drop(columns='Delinquent90')
y = df['Delinquent90'].astype("float")

In [ ]:
X = X.fillna(X.mean())

In [ ]:
X = X.values.astype("float32")
y = y.values.reshape(-1, 1).astype("float32")


In [ ]:
X_train, X_rem, y_train, y_rem = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_rem, y_rem, test_size=0.5, random_state=42, stratify=y_rem)

нормализуем

In [ ]:
mean = X_train.mean(axis=0)
std = X_train.std(axis=0) + 1e-6

X_train = (X_train - mean) / std
X_val   = (X_val   - mean) / std
X_test  = (X_test  - mean) / std

делаем тензоры

In [ ]:
device = "cpu"
X_train_t = torch.tensor(X_train, device=device)
y_train_t = torch.tensor(y_train, device=device)
X_val_t   = torch.tensor(X_val,   device=device)
y_val_t   = torch.tensor(y_val,   device=device)
X_test_t  = torch.tensor(X_test,  device=device)
y_test_t  = torch.tensor(y_test,  device=device)

In [ ]:
samples, features = X_train_t.shape
samples, features

(60000, 9)

помним что в этой лабораторной у нас присутствует очень сильный дисбаланс

In [ ]:
y.mean()

np.float32(0.06684)

примерно 94% - 0 и 6% - 1

## Делаем логистическую регрессию

пишем свои функции и метрики

сигмойда

In [ ]:
def sigmoid(x):
    return 1.0 / (1.0 + torch.exp(-x))

бинарная кросс-энтропия(Лосс)

In [ ]:
def binary_cross_entropy(y_pred_prob, y_true):
    eps = 1e-7
    y_pred_prob = torch.clamp(y_pred_prob, eps, 1 - eps)
    loss = -(y_true * torch.log(y_pred_prob) +
             (1 - y_true) * torch.log(1 - y_pred_prob))
    return loss.mean()

прямой проход

In [ ]:
def forward_logistic(X):
    logits = X @ W + b
    probs  = sigmoid(logits)
    return probs

In [ ]:
def accuracy(y_pred_prob, y_true, threshold=0.5):
    y_pred = (y_pred_prob >= threshold).float()
    return (y_pred == y_true).float().mean().item()

Параметры модели:


In [ ]:
torch.manual_seed(0)
W = torch.randn(features, 1, device=device, requires_grad=True)
W.data *= 0.01
b = torch.zeros(1, device=device, requires_grad=True)

In [ ]:
def train_logistic(
    X_train, y_train,
    X_val, y_val,
    lr=0.1,
    epochs=5,
    batch_size=1024
):
    global W, b

    n_train = X_train.shape[0]

    for epoch in range(1, epochs + 1):
        perm = torch.randperm(n_train)
        total_loss = 0.0

        for i in range(0, n_train, batch_size):
            idx = perm[i:i + batch_size]
            xb = X_train[idx]
            yb = y_train[idx]
            probs = forward_logistic(xb)
            loss = binary_cross_entropy(probs, yb)
            loss.backward()

            with torch.no_grad():
                W.data -= lr * W.grad
                b.data -= lr * b.grad

            W.grad.zero_()
            b.grad.zero_()
            total_loss += loss.item() * xb.size(0)

        with torch.no_grad():
            train_probs = forward_logistic(X_train)
            val_probs   = forward_logistic(X_val)
            train_loss  = binary_cross_entropy(train_probs, y_train).item()
            val_loss    = binary_cross_entropy(val_probs,   y_val).item()
            train_acc   = accuracy(train_probs, y_train)
            val_acc     = accuracy(val_probs,   y_val)

        print(f"Epoch {epoch}: train_loss={train_loss:.4f}, val_loss={val_loss:.4f}, train_acc={train_acc:.4f}, val_acc={val_acc:.4f}")

In [ ]:
train_logistic(X_train_t, y_train_t, X_val_t, y_val_t, lr=0.1, epochs=5)

Epoch 1: train_loss=0.3076, val_loss=0.3067, train_acc=0.9331, val_acc=0.9337
Epoch 2: train_loss=0.2586, val_loss=0.2574, train_acc=0.9331, val_acc=0.9337
Epoch 3: train_loss=0.2455, val_loss=0.2440, train_acc=0.9331, val_acc=0.9337
Epoch 4: train_loss=0.2406, val_loss=0.2390, train_acc=0.9331, val_acc=0.9337
Epoch 5: train_loss=0.2384, val_loss=0.2369, train_acc=0.9329, val_acc=0.9337


Epoch 1:

- train_loss=0.2373, val_loss=0.2357, train_acc=0.9329, val_acc=0.9335

Epoch 2:
- train_loss=0.2366, val_loss=0.2349, train_acc=0.9330, val_acc=0.9336

Epoch 3:
- train_loss=0.2362, val_loss=0.2345, train_acc=0.9330, val_acc=0.9336

Epoch 4:
- train_loss=0.2360, val_loss=0.2342, train_acc=0.9330, val_acc=0.9336

Epoch 5:
- train_loss=0.2358, val_loss=0.2339, train_acc=0.9330, val_acc=0.9336

## заделаем MLP
на 2 слоя

In [ ]:
torch.manual_seed(0)

hidden_size = 32

поставил 32

In [ ]:
W1 = torch.randn(features, hidden_size, device=device, requires_grad=True)
W1.data *= 0.01
b1 = torch.zeros(hidden_size, device=device, requires_grad=True)

W2 = torch.randn(hidden_size, 1, device=device, requires_grad=True)
W2.data *= 0.01
b2 = torch.zeros(1, device=device, requires_grad=True)

In [ ]:
def relu(x):
    return torch.clamp(x, min=0.0)

In [ ]:
def forward_mlp(X):
    hidden = relu(X @ W1 + b1)
    logits = hidden @ W2 + b2
    probs  = sigmoid(logits)
    return probs

In [ ]:
def train_mlp(
    X_train, y_train,
    X_val, y_val,
    lr=0.01,
    epochs=10,
    batch_size=1024
):
    global W1, b1, W2, b2

    n_train = X_train.shape[0]

    for epoch in range(1, epochs + 1):
        perm = torch.randperm(n_train)
        total_loss = 0.0

        for i in range(0, n_train, batch_size):
            idx = perm[i:i + batch_size]
            xb = X_train[idx]
            yb = y_train[idx]

            probs = forward_mlp(xb)
            loss = binary_cross_entropy(probs, yb)
            loss.backward()

            with torch.no_grad():
                W1.data -= lr * W1.grad
                b1.data -= lr * b1.grad
                W2.data -= lr * W2.grad
                b2.data -= lr * b2.grad

            W1.grad.zero_()
            b1.grad.zero_()
            W2.grad.zero_()
            b2.grad.zero_()

            total_loss += loss.item() * xb.size(0)

        with torch.no_grad():
            train_probs = forward_mlp(X_train)
            val_probs   = forward_mlp(X_val)
            train_loss  = binary_cross_entropy(train_probs, y_train).item()
            val_loss    = binary_cross_entropy(val_probs,   y_val).item()
            train_acc   = accuracy(train_probs, y_train)
            val_acc     = accuracy(val_probs,   y_val)

        print(f"[MLP] Epoch {epoch}: train_loss={train_loss:.4f}, val_loss={val_loss:.4f}, train_acc={train_acc:.4f}, val_acc={val_acc:.4f}")

In [ ]:
train_mlp(X_train_t, y_train_t, X_val_t, y_val_t, lr=0.01, epochs=10)

[MLP] Epoch 1: train_loss=0.5969, val_loss=0.5969, train_acc=0.9332, val_acc=0.9331
[MLP] Epoch 2: train_loss=0.5249, val_loss=0.5250, train_acc=0.9332, val_acc=0.9331
[MLP] Epoch 3: train_loss=0.4707, val_loss=0.4707, train_acc=0.9332, val_acc=0.9331
[MLP] Epoch 4: train_loss=0.4292, val_loss=0.4293, train_acc=0.9332, val_acc=0.9331
[MLP] Epoch 5: train_loss=0.3971, val_loss=0.3972, train_acc=0.9332, val_acc=0.9331
[MLP] Epoch 6: train_loss=0.3719, val_loss=0.3720, train_acc=0.9332, val_acc=0.9331
[MLP] Epoch 7: train_loss=0.3518, val_loss=0.3519, train_acc=0.9332, val_acc=0.9331
[MLP] Epoch 8: train_loss=0.3357, val_loss=0.3358, train_acc=0.9332, val_acc=0.9331
[MLP] Epoch 9: train_loss=0.3225, val_loss=0.3226, train_acc=0.9332, val_acc=0.9331
[MLP] Epoch 10: train_loss=0.3117, val_loss=0.3118, train_acc=0.9332, val_acc=0.9331


Функции активации -

На выходе у модели стоит сигмоида. Она сжимает результат в интервал от 0 до 1, что удобно. Интерпретировал это значение как вероятность того, что клиент окажется неплатежеспособным (Delinquent90=1). Условно бинарное значение ввел  

В скрытых слоях я использовал функцию ReLU. Простая, эффективная и позволяет градиентам нормально распространяться при обучении (показывает себя хорошо на табличных данных), в отличие от сигмоид.

В качестве функции потерь использовал бинарную кросс-энтропию. Насколько наши прогнозы вероятностей отличаются от реальных меток.


Особенность функции в том, что она очень сильно наказывает модель за уверенные, но неправильные предсказания. Например, если модель на 99% была уверена, что клиент платёжеспособен, а он ушёл — это будет считаться большой ошибкой. В кредитном скоринге это критически важно, потому что цена ошибки может дорого стоить.

Логистическая регрессия

Лосс падает — модель учится.

Accuracy 93% — но это из-за несбалансированных данных (93% класса 0).

Вероятности в целом становятся лучше.

MLP:

Лосс хуже, чем у регрессии (~0.31 против ~0.24).

Accuracy аналогично.

К сожалению усложнение модели не дало преимущества.


Логистическая регрессия работает лучше на этих данных. Переобучения нет. Для полноты анализа нужно смотреть не только accuracy, но и другие метрики (ROC-AUC, precision/recall).